Notebook dealing with the final preprocessing steps for the neural networks.


In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.utils.data
import pandas as pd
from torch.utils import data
from numpy import array
from numpy import argmax
import argparse
from torch.autograd import Variable
from torch import optim
import numpy as np
import os
#from sklearn.model_selection import train_test_split
import logging
import pickle as pkl
import warnings
pd.options.mode.chained_assignment = None
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="4"
import glob
import random
random.seed(1991)
#torch.set_default_tensor_type('torch.cuda.DoubleTensor')

In [2]:
torch.cuda.is_available()

True

In [3]:
torch.manual_seed(1991)
if not torch.cuda.is_available():
    
        print("WARNING: You have a CUDA device, so you should probably run with --cuda")

device = torch.device("cuda")

In [4]:
heads=pd.read_csv("/data/dharp/compounds/datasets/v2_aware/heads_reduced.csv",sep="\t",header=None)
heads=heads.drop(1,axis=1).groupby(0).mean()
heads.columns=list(range(0,300))
heads.index.name="head"
heads.reset_index(inplace=True)
heads

,head,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
0,#,0.052947,-0.043277,-0.048858,-0.005336,0.002371,0.024603,0.011224,0.004311,0.017362,...,0.005057,0.015608,0.034102,0.021763,0.008789,0.011844,0.008573,-0.019235,-0.000776,-0.008474
1,%,0.124636,-0.108861,-0.031280,0.030882,0.005079,0.024689,-0.001271,0.021538,0.009361,...,0.120386,0.056511,0.084156,0.044476,-0.063122,-0.050190,0.047394,-0.033286,-0.006415,0.198380
2,',0.096203,-0.088409,-0.133258,-0.015924,0.004648,-0.049040,0.030154,0.165385,0.008845,...,0.047555,0.038746,0.025380,0.000881,0.023599,-0.018918,0.008246,-0.046232,-0.054471,-0.044855
3,'s,0.026463,-0.024737,-0.032018,-0.003708,-0.000935,0.002239,0.017015,0.003898,0.015470,...,-0.052857,0.005489,-0.011554,0.014019,-0.004140,-0.032454,-0.034887,0.021859,0.010536,-0.053386
4,*,0.119781,-0.110382,-0.137271,-0.002352,-0.005441,-0.031903,0.020383,0.120265,0.006734,...,-0.016877,0.018148,0.011201,0.058570,-0.011987,-0.056249,-0.027524,0.035355,0.015572,-0.105566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22744,®,0.095624,-0.081476,-0.100241,-0.012479,-0.002418,0.003100,-0.016008,0.000561,-0.002791,...,-0.026871,-0.026152,-0.026665,-0.013554,-0.006794,0.053076,-0.073821,-0.000033,0.006830,-0.117985
22745,°,0.079986,-0.067950,-0.016365,0.057317,0.002860,0.042968,-0.007963,0.010572,-0.001698,...,0.081680,-0.046020,-0.031310,-0.094081,-0.020666,0.001451,0.094387,-0.035220,-0.005892,0.035907
22746,±,0.075897,-0.063193,-0.060545,0.017133,0.001398,0.017076,-0.022170,0.003148,-0.009074,...,-0.014200,0.012466,0.026461,-0.004472,-0.009574,0.022614,-0.004230,0.004782,-0.016364,0.014534
22747,•,0.038490,-0.031375,-0.042233,-0.005843,0.000251,0.012283,-0.021439,-0.000599,-0.001301,...,-0.052987,-0.062050,-0.111422,-0.113398,-0.061910,-0.008103,-0.014199,-0.050712,-0.052957,-0.141861


In [5]:
modifiers=pd.read_csv("/data/dharp/compounds/datasets/v2_aware/modifiers_reduced.csv",sep="\t",header=None)
modifiers=modifiers.drop(1,axis=1).groupby(0).mean()
modifiers.columns=list(range(0,300))
modifiers.index.name="modifier"
modifiers.reset_index(inplace=True)
modifiers

,modifier,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
0,#,0.070692,-0.062718,-0.025479,0.004098,0.005732,0.096587,-0.023396,0.002303,-0.034061,...,-0.026612,-0.027087,0.017672,0.001767,0.014812,-0.002746,-0.013836,0.014384,0.032302,-0.034401
1,%,0.158867,-0.113833,0.078792,0.009123,0.002792,0.052180,-0.007997,0.001448,-0.019802,...,0.029924,-0.063970,-0.036576,0.090016,-0.008387,0.014423,-0.001059,0.023197,-0.002569,0.055429
2,',0.164654,-0.145436,-0.222945,-0.030129,-0.014309,-0.071962,0.041732,0.175078,0.010543,...,0.057181,0.021279,0.055763,0.061123,-0.057950,-0.027282,0.039698,-0.072258,-0.030215,-0.017170
3,*,0.062809,-0.046161,-0.036354,-0.004650,0.010484,0.146974,0.103799,0.086697,-0.126842,...,-0.009160,0.016757,0.012849,0.016077,-0.038977,-0.014151,-0.010778,0.013873,-0.006668,-0.037839
4,+,0.090124,-0.081503,-0.062543,-0.000957,0.001116,0.049211,0.046728,0.008222,-0.014001,...,-0.008780,0.005435,0.027809,0.026495,-0.018012,-0.000396,-0.024420,0.013172,0.001498,-0.001563
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21104,°,0.028880,-0.025652,-0.006227,0.000458,0.001736,0.025359,0.006190,0.001714,0.000403,...,0.002930,-0.000385,0.001108,-0.001374,0.000402,0.005247,0.000652,-0.000930,-0.001014,0.006796
21105,±,0.084076,-0.074522,-0.069984,0.011513,-0.000256,0.026765,0.012150,-0.001651,-0.001663,...,0.018787,-0.014928,0.056103,0.028166,0.035033,-0.071992,-0.025501,-0.057028,-0.096342,0.006603
21106,•,0.173109,-0.153092,-0.123386,-0.018108,0.008101,0.107705,-0.071794,0.073006,0.003101,...,-0.042510,0.023166,0.056634,-0.004273,-0.032277,0.017683,0.004376,0.023457,0.073930,0.037702
21107,™,0.026644,-0.026715,-0.046124,-0.006482,-0.001864,-0.002104,-0.007275,-0.002484,0.000462,...,-0.017913,0.065967,0.007226,0.054388,-0.050637,0.018457,0.047483,-0.016264,-0.038279,0.043835


In [6]:
with open('/data/dharp/compounds/datasets/compound_testset.txt','r') as f:
    novel_compounds_list=f.read().split("\n")
novel_compounds_list=novel_compounds_list[:-1]
len(novel_compounds_list)

12572

novel_compounds_list = pkl.load( open( "/data/dharp/compounding/datasets/novel_compounds_list.pkl", "rb" ) )
m, h = zip(*novel_compounds_list)
heads_list=list(set(h))
modifiers_list=list(set(m))

In [7]:
novel_compounds=pd.DataFrame(novel_compounds_list)
novel_compounds[['modifier','head']]=novel_compounds[0].str.split(" ",expand=True)
novel_compounds.drop(0,axis=1,inplace=True)
novel_compounds.sample(30)

,modifier,head
8624,policy,diffusion
3302,duplex,loop
10456,small,trends
9602,retirement,trends
12419,world,hitler
3434,education,relevance
8565,plasma,screen
777,asylum,rights
1898,classics,cosimo
332,agency,cue


In [8]:
positive_heads=pd.merge(novel_compounds.drop('modifier',axis=1),heads,on=["head"],how='left')

positive_modifiers=pd.merge(novel_compounds.drop('head',axis=1),modifiers,on=["modifier"])
#positive_df['Plausibility']=True

In [9]:
to_remove=positive_heads[positive_heads.isna().any(axis=1)].index.values[0]
to_remove

10099

In [10]:
positive_modifiers.drop(to_remove,inplace=True)
positive_heads.drop(to_remove,inplace=True)
novel_compounds.drop(to_remove,inplace=True)

In [11]:
assert positive_modifiers.shape[0]==positive_heads.shape[0]==novel_compounds.shape[0]

In [12]:
positive_heads_tensor = torch.tensor(positive_heads.drop('head',axis=1).values)
positive_heads_tensor.shape

torch.Size([12571, 300])

In [13]:
positive_modifiers_tensor = torch.tensor(positive_modifiers.drop('modifier',axis=1).values)
positive_modifiers_tensor.shape

torch.Size([12571, 300])

In [14]:
positive_Y=torch.ones(positive_modifiers_tensor.shape[0])
positive_Y.shape

torch.Size([12571])

In [15]:
positive_class=torch.cat((positive_modifiers_tensor, positive_heads_tensor), 1)
positive_class.shape

torch.Size([12571, 600])

In [33]:
def neg_df_creator(file):
    pkl_file=pkl.load( open(file,'rb'))
    df=pd.DataFrame(pkl_file)
    
    df.columns=['modifier','head']
    negative_heads=pd.merge(df.drop('modifier',axis=1),heads,on=["head"],how='left')
    negative_modifiers=pd.merge(df.drop('head',axis=1),modifiers,on=["modifier"],how='left')
    
    to_remove_mods=negative_modifiers[negative_modifiers.isna().any(axis=1)].index.to_list()
    to_remove_heads=negative_heads[negative_heads.isna().any(axis=1)].index.to_list()
    
    negative_modifiers.drop(to_remove_mods+to_remove_heads,inplace=True)
    negative_heads.drop(to_remove_mods+to_remove_heads,inplace=True)
    negative_heads_tensor = torch.tensor(negative_heads.drop('head',axis=1).values)
    negative_modifiers_tensor = torch.tensor(negative_modifiers.drop('modifier',axis=1).values)

    negative_Y=torch.zeros(negative_modifiers_tensor.shape[0])
    negative_class=torch.cat((negative_modifiers_tensor, negative_heads_tensor), 1)

    return negative_class,negative_Y,df

In [62]:
def tensor_joiner(files):
    tensor_list=[]
    df_list=[]
    for file in files:
        negative_class,negative_Y,neg_df=neg_df_creator(file)
        X=torch.cat((positive_class, negative_class), 0)
        Y=torch.cat((positive_Y,negative_Y),0)
        tensor_list.append([X,Y])
        df_list.append(neg_df)
    
    return tensor_list,df_list

In [63]:
corrupt_modifier_files=[]
for file in glob.glob("/data/dharp/compounds/datasets/corrupt_modifier*"):
    corrupt_modifier_files.append(file)
corrupt_modifiers,corrupt_modifiers_df=tensor_joiner(corrupt_modifier_files)

In [ ]:
analysis	expert
privity	channel
player	educator
chocolate	books
flower	decree
treaty	discipline
storm	profile
botanical	store
wiener	weight
diploma	services
nhs	dispute
shock	mobility
glacier	banking
commercial	toxicity
sociometric	system
nightshade	balance
enlightened	specification



coffee	previn
city	changer
subscription	guilt
admission	box
brain	arbitration
volunteer	dancer
manor	woods
study	tantras
organizational	geographe

In [111]:
corrupt_heads_df[1].sample(30)


,modifier,head
6081,kinase,developer
1777,chinese,pancreas
10910,study,tantras
12554,•,handbook
11669,triple,norm
974,base,discourse
9816,s.,manager
6398,line,abragam
11505,toy,vac
7911,online,verification


In [64]:
corrupt_head_files=[]
for file in glob.glob("/data/dharp/compounds/datasets/corrupt_head*"):
    corrupt_head_files.append(file)
corrupt_heads,corrupt_heads_df=tensor_joiner(corrupt_head_files)

In [65]:
input_size = 600
hidden_size = 300
num_classes = 2
num_epochs = 50
batch_size = 100
learning_rate = 0.001

In [73]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes,bias=False)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes).cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()
        # Linear function
        self.fc1 = nn.Linear(input_dim, hidden_dim) 

        # Non-linearity
        self.sigmoid = nn.Sigmoid()

        # Linear function (readout)
        self.fc2 = nn.Linear(hidden_dim, output_dim,bias=False)

    def forward(self, x):
        # Linear function  # LINEAR
        out = self.fc1(x)

        # Non-linearity  # NON-LINEAR
        out = self.sigmoid(out)

        # Linear function (readout)  # LINEAR
        out = self.fc2(out)
        return out

In [91]:
def looper(datasets):
    total_accuracy=[]
    j=0
    for dataset in datasets:
        X=dataset[0]
        Y=dataset[1]
        n = len(X)  # how many total elements you have
        n_test = int( n * .2 )  # number of test/val elements
        n_train = n - n_test
        idx = list(range(n))  # indices to all elements
        random.shuffle(idx)  # in-place shuffle the indices to facilitate random splitting
        train_idx = idx[:n_train]
        test_idx = idx[n_train:]
        trX=X[train_idx].float().to(device)
        teX=X[test_idx].float().to(device)

        trY=Y[train_idx].long().to(device)
        teY=Y[test_idx].long().to(device)
        #model = NeuralNet(input_size, hidden_size, num_classes).to(device)
        #criterion = nn.CrossEntropyLoss()
        #optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        n_examples=trX.shape[0]
        print(trX)
        print(trY)
        for i in range(num_epochs):

            cost = 0.

            num_batches = n_examples // batch_size
            for k in range(num_batches):
                optimizer.zero_grad()
                start, end = k * batch_size, (k + 1) * batch_size
                outputs = model(trX[start:end])
                loss = criterion(outputs, trY[start:end])

                loss.backward()
                optimizer.step()


            #if (k+1) % 100 == 0:
            #print ('Epoch [{}/{}], Loss: {:.4f}'.format(i+1, num_epochs, loss.item()))

        with torch.no_grad():
            correct=0
            total=0
            outputs = model(teX)
            score, predicted = torch.max(outputs.data, 1)
            total += teY.size(0)
            correct += (predicted == teY).sum().item()
        curr_acc=100 * correct / total
        print(curr_acc)
        j+=1
        return score,predicted
        total_accuracy.append(curr_acc)
        predicted_compounds=pd.DataFrame()
        predicted_compounds['score']=score.cpu().data.numpy()
        predicted_compounds['predicted']=predicted.cpu().data.numpy()
        predicted_compounds['teY']=teY.cpu().data.numpy()
        predicted_compounds.to_csv(f'/data/dharp/compounds/datasets/mvp{j}_corrhead_actual_pred.csv',index=False,header=True,sep="\t")
        
    return total_accuracy
        

In [92]:
a,b=looper(corrupt_heads)

tensor([[ 0.1471, -0.1360, -0.1836,  ..., -0.0231,  0.0068, -0.0387],
        [ 0.0399, -0.0330, -0.0511,  ...,  0.0154,  0.0253,  0.0530],
        [ 0.3641, -0.3303, -0.4455,  ...,  0.0223, -0.0232,  0.1361],
        ...,
        [ 0.1605, -0.1243,  0.0024,  ...,  0.0028, -0.0068,  0.0324],
        [ 0.0306, -0.0241, -0.0422,  ...,  0.0137, -0.0214, -0.0067],
        [ 0.0927, -0.0612, -0.0694,  ...,  0.0561,  0.0355, -0.0845]],
       device='cuda:0')
tensor([1, 0, 0,  ..., 0, 0, 1], device='cuda:0')


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [86]:
len(corrupt_heads[0][1])

25117

In [83]:
corrupt_heads[0][1].shape

torch.Size([25117])

In [80]:
corrupt_heads[1]

[tensor([[ 0.0707, -0.0627, -0.0255,  ..., -0.0521, -0.0351, -0.1222],
         [ 0.0707, -0.0627, -0.0255,  ..., -0.0089,  0.0050, -0.0559],
         [ 0.0707, -0.0627, -0.0255,  ..., -0.0041, -0.0021,  0.0047],
         ...,
         [ 0.1731, -0.1531, -0.1234,  ...,  0.0100,  0.1263,  0.1329],
         [ 0.1731, -0.1531, -0.1234,  ...,  0.0018,  0.0112,  0.0224],
         [ 0.0266, -0.0267, -0.0461,  ...,  0.0572,  0.1193, -0.0121]],
        dtype=torch.float64),
 tensor([1., 1., 1.,  ..., 0., 0., 0.])]

In [110]:
cor_mod_acc=looper(corrupt_modifiers)

71.66732988460008
71.48258706467662
70.97095105451652
70.14331210191082
70.40796019900498
70.69239952248309
71.96019900497512
71.04477611940298
71.22388059701493
70.55312375646638


In [76]:
corrupt_modifiers[0]

[tensor([[ 0.0707, -0.0627, -0.0255,  ..., -0.0521, -0.0351, -0.1222],
         [ 0.0707, -0.0627, -0.0255,  ..., -0.0089,  0.0050, -0.0559],
         [ 0.0707, -0.0627, -0.0255,  ..., -0.0041, -0.0021,  0.0047],
         ...,
         [ 0.2240, -0.2052, -0.2706,  ...,  0.0389, -0.0236, -0.0027],
         [ 0.0483, -0.0472, -0.0643,  ..., -0.0330,  0.0514, -0.0913],
         [ 0.0516, -0.0436, -0.0342,  ...,  0.0018,  0.0102, -0.0190]],
        dtype=torch.float64),
 tensor([1., 1., 1.,  ..., 0., 0., 0.])]

82.91589770630108
83.12681254943317
82.59952544160295
82.76649969241585
83.14438878636084
83.05650760172247
82.90710958783724
83.22348185253537
82.67861850777749
82.74892345548818

In [111]:
round(np.mean(cor_head_acc),2)

74.18

In [112]:
round(np.std(cor_head_acc),2)

0.48

In [27]:
cor_mod_acc=looper(corrupt_modifiers)

82.50285613850075
82.6610422708498
82.50285613850075
81.52737498901485
83.08287195711398
82.27436505844099
83.14438878636084
82.4677036646454
83.06529572018631
82.26557693997715


82.88074523244573
83.40803234027595
82.29194129536866
81.87889972756832
82.72255910009667
82.21284822919412
82.45891554618156
82.5643729677476
82.6610422708498
82.3007294138325

In [113]:
round(np.mean(cor_mod_acc),2)

71.01

In [114]:
round(np.std(cor_mod_acc),2)

0.55

In [117]:
novel_compounds

,modifier,head
0,#,husky
1,#,nat
2,#,new
3,#,region
4,#,text
...,...,...
12567,•,switzerland
12568,•,transportation
12569,•,type
12570,•,water
